In [2]:
import requests
import json
import pandas as pd
import ta

In [3]:
#Daily Timeframe Data Grabber
def daily_df(ticker):

    API_URL = "https://www.alphavantage.co/query"

    data = {
        "function": "TIME_SERIES_DAILY",
        "symbol": ticker,
        "outputsize": "compact",
        "datatype": "json",
        "apikey": "PVGTC9P6IUS7XLQ9",
        }

    response = requests.get(API_URL, params=data)
    
    if response.status_code == 200:
        stock_data = response.json()
        meta_data = stock_data['Meta Data']
        daily_data = stock_data['Time Series (Daily)']
        daily_dataframe_rev = pd.DataFrame.from_dict(daily_data)
        daily_dataframe = daily_dataframe_rev.transpose()     
        daily_dataframe.columns = ['open', 'high', 'low', 'close', 'volume']
        df_entries2float(daily_dataframe)
        return daily_dataframe

In [4]:
#Converting dataframe entries to floats
def df_entries2float(df):
    for i in df.columns:
        column2convert = df[i]
        column_size = column2convert.size
        for k in range(0,column_size):
            column2convert[k] = float(column2convert[k])
    return df

In [5]:
#Reversing dataframe
def rev(df):
    return df.sort_index(ascending = True, axis = 0)

In [6]:
#Running TA with proper formatting
def tech_indicator(indicator, daily_df):
    indicator_df = pd.DataFrame(indicator)
    indicator_df = indicator_df.sort_index(ascending=False, axis=0)
    resulting_df = daily_df.join(indicator_df)
    return resulting_df

In [7]:
#All technical indicators in use
def run_indicators(df):
    high = rev(df['high'])
    low = rev(df['low'])    
    close = rev(df['close'])
    volume = rev(df['volume'])
    
    df = tech_indicator(ta.momentum.wr(high,low,close),df)
    df = tech_indicator(ta.momentum.money_flow_index(high,low,close,volume),df)
    df = tech_indicator(ta.momentum.stoch_signal(high,low,close),df)
    df = tech_indicator(ta.momentum.tsi(close),df)
    df = tech_indicator(ta.trend.macd(close),df)
    df = tech_indicator(ta.trend.trix(close),df)
    df = tech_indicator(ta.trend.aroon_up(close),df)
    df = tech_indicator(ta.trend.aroon_down(close),df)
    df = tech_indicator(ta.volatility.keltner_channel_hband_indicator(high,low,close),df)
    df = tech_indicator(ta.volatility.keltner_channel_lband_indicator(high,low,close),df)
    df = tech_indicator((ta.volatility.bollinger_hband_indicator(close)),df)
    df = tech_indicator((ta.volatility.bollinger_lband_indicator(close)),df)  
    df = tech_indicator(ta.volume.chaikin_money_flow(high,low,close,volume),df)
    
    df['bbi'] = df['bbihband'].subtract(df['bbilband'])
    df['kci'] = df['kci_hband'].subtract(df['kci_lband'])
    del df['bbihband']
    del df['bbilband']
    del df['kci_hband']
    del df['kci_lband']
    df = tech_indicator(ta.momentum.rsi(close),df)
    
    return df

In [14]:
#Mapping values
def map_values(df):
    length = len(df)
    
    df = df.copy()
    
    #Mapping wr values
    wr = df.columns.get_loc('wr')
    for i in range(length):
        if -100 <= df.iloc[i,wr] <= -80:
            df.iloc[i,wr] = 2
        elif -20<= df.iloc[i,wr] <= 0:
            df.iloc[i,wr] = -2
        elif -30 <= df.iloc[i,wr] < -20:
            df.iloc[i,wr] = -1
        elif -80 < df.iloc[i,wr] <= -70:
            df.iloc[i,wr] = 1
        else:
            df.iloc[i,wr] = 0
            
    #Mapping MFI values
    mfi = df.columns.get_loc('mfi_14')
    for i in range(length):
        if 0 <= df.iloc[i,mfi] <= 20:
            df.iloc[i,mfi] = 2
        elif 80 <= df.iloc[i,mfi] <= 100:
            df.iloc[i,mfi] = -2
        elif 20 <= df.iloc[i,mfi] < 30:
            df.iloc[i,mfi] = 1
        elif 70 < df.iloc[i,mfi] <= 80:
            df.iloc[i,mfi] = -1
        else:
            df.iloc[i,mfi] = 0
            
    #Mapping Stoch_d values
    stoch = df.columns.get_loc('stoch_d')
    for i in range(length):
        if 80 <= df.iloc[i,stoch] <= 100:
            df.iloc[i,stoch] = -2
        elif 0 <= df.iloc[i,stoch] <= 20:
            df.iloc[i,stoch] = 2
        elif 20 < df.iloc[i,stoch] <= 30:
            df.iloc[i,stoch] = 1
        elif 70 <= df.iloc[i,stoch] < 80:
            df.iloc[i,stoch] = -1
        else:
            df.iloc[i,stoch] = 0
    
    #Mapping TSI values
    tsi = df.columns.get_loc('tsi')
    for i in range(length):
        if df.iloc[i,tsi] < -25:
            df.iloc[i,tsi] = -2
        elif df.iloc[i,tsi] <= -10:
            df.iloc[i,tsi] = -1
        elif df.iloc[i,tsi] <= 10:
            df.iloc[i,tsi] = 0
        elif df.iloc[i,tsi] < 25:
            df.iloc[i,tsi] = 1
        else:
            df.iloc[i,tsi] = 2
    
    #Mapping MACD values
    macd = df.columns.get_loc('MACD_12_26')
    for i in range(length):
        if 0 < df.iloc[i,macd]:
            df.iloc[i,macd] = 1
        elif df.iloc[i,macd] < 0:
            df.iloc[i,macd] = -1
        else:
            df.iloc[i,macd] = 0
            
    #Mapping TRIX values
    trix = df.columns.get_loc('trix_15')
    for i in range(length):
        if 0 < df.iloc[i,trix]:
            df.iloc[i,trix] = 1
        elif df.iloc[i,trix] < 0:
            df.iloc[i,trix] = -1
        else:
            df.iloc[i,trix] = 0
            
    #Mapping Aroon values
    values = []
    aroon_up = df.columns.get_loc('aroon_up25')
    aroon_down = df.columns.get_loc('aroon_down25')
    for i in range(length):
        if df.iloc[i,aroon_up] >= 70 and df.iloc[i,aroon_down] <= 30:
            values.append(2.0)
        elif df.iloc[i,aroon_down] >= 70 and df.iloc[i,aroon_up] <= 30:
            values.append(-2.0)
        elif df.iloc[i,aroon_up] > df.iloc[i,aroon_down]:
            values.append(1.0)
        elif df.iloc[i,aroon_down] > df.iloc[i,aroon_up]:
            values.append(-1.0)
        else:
            values.append(0.0)
    df['aroon'] = values
    
    #Mapping Chaikin values
    cmf = df.columns.get_loc('cmf')
    for i in range(length):
        if 0.05 <= df.iloc[i,cmf] < 0.1:
            df.iloc[i,cmf] = 1
        elif -0.1 < df.iloc[i,cmf] <= -0.05:
            df.iloc[i,cmf] = -1
        elif df.iloc[i,cmf] >= 0.1:
            df.iloc[i,cmf] = 2
        elif df.iloc[i,cmf] <= -0.1:
            df.iloc[i,cmf] = -2
        else:
            df.iloc[i,cmf] = 0
    
    #Mapping rsi values
    rsi = df.columns.get_loc('rsi')
    for i in range(length):
        if 70 <= df.iloc[i,rsi] < 100:
            df.iloc[i,rsi] = -2
        elif 0 <= df.iloc[i,rsi] <= 30:
            df.iloc[i,cmf] = 2
        elif 30 < df.iloc[i,rsi] >= 40:
            df.iloc[i,rsi] = 1
        elif 60 <= df.iloc[i,rsi] < 70:
            df.iloc[i,rsi] = -1
        else:
            df.iloc[i,rsi] = 0
            
    del df['aroon_up25']
    del df['aroon_down25']

    return df

In [15]:
test3 = map_values(test2)
test3.head(30)

,open,high,low,close,volume,wr,mfi_14,stoch_d,tsi,MACD_12_26,trix_15,cmf,bbi,kci,rsi,aroon
2019-10-25,243.16,246.72,242.89,246.58,1.58277e+07,-2,-1.0,-2.0,1.0,1.0,1.0,2.0,0.0,0.0,-2.0,2.0
2019-10-24,244.51,244.8,241.81,243.58,1.73188e+07,-2,-1.0,-2.0,1.0,1.0,1.0,1.0,0.0,0.0,-2.0,2.0
2019-10-23,242.1,243.24,241.22,243.18,1.89572e+07,-2,-2.0,-2.0,1.0,1.0,1.0,1.0,0.0,0.0,-2.0,2.0
2019-10-22,241.16,242.2,239.62,239.96,2.05734e+07,-2,-1.0,-2.0,1.0,1.0,1.0,1.0,0.0,0.0,-2.0,2.0
2019-10-21,237.52,240.99,237.32,240.51,2.18118e+07,-2,0.0,-2.0,1.0,1.0,1.0,0.0,0.0,0.0,-2.0,2.0
2019-10-18,234.59,237.58,234.29,236.41,2.43584e+07,-2,0.0,-2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,2.0
2019-10-17,235.09,236.15,233.52,235.28,1.68963e+07,-2,0.0,-2.0,1.0,1.0,1.0,-1.0,0.0,0.0,1.0,1.0
2019-10-16,233.37,235.24,233.2,234.37,1.84758e+07,-2,0.0,-2.0,1.0,1.0,1.0,-2.0,0.0,0.0,1.0,1.0
2019-10-15,236.39,237.65,234.88,235.32,2.184e+07,-2,0.0,-2.0,1.0,1.0,1.0,-1.0,0.0,0.0,1.0,1.0
2019-10-14,234.9,238.13,234.67,235.87,2.41069e+07,-2,0.0,-2.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,2.0


In [9]:
test = daily_df('AAPL')
test2 = run_indicators(test)
test2.head(50)

,open,high,low,close,volume,wr,mfi_14,stoch_d,tsi,MACD_12_26,trix_15,aroon_up25,aroon_down25,cmf,bbi,kci,rsi
2019-10-25,243.16,246.72,242.89,246.58,1.58277e+07,-0.625279,78.839371,97.701567,22.802090,6.642883,0.368968,100.0,8.0,0.113703,0.0,0.0,76.654622
2019-10-24,244.51,244.8,241.81,243.58,1.73188e+07,-5.95994,79.677086,95.151713,21.727455,6.315888,0.354970,100.0,12.0,0.075124,0.0,0.0,74.179051
2019-10-23,242.1,243.24,241.22,243.18,1.89572e+07,-0.310078,80.580307,96.519644,20.817717,6.111974,0.342188,100.0,16.0,0.068915,0.0,0.0,73.835540
2019-10-22,241.16,242.2,239.62,239.96,2.05734e+07,-8.27484,73.420132,94.130249,19.847619,5.803504,0.329285,96.0,20.0,0.067274,0.0,0.0,70.946320
2019-10-21,237.52,240.99,237.32,240.51,2.18118e+07,-1.85615,65.350606,92.758095,19.076799,5.653055,0.317651,100.0,24.0,0.046567,0.0,0.0,72.211041
2019-10-18,234.59,237.58,234.29,236.41,2.43584e+07,-7.47826,66.382417,87.927536,18.145010,5.322821,0.305483,100.0,28.0,0.015462,0.0,0.0,68.299301
2019-10-17,235.09,236.15,233.52,235.28,1.68963e+07,-12.3913,66.399566,86.347826,17.504198,5.241107,0.295093,84.0,32.0,-0.086985,0.0,0.0,67.114594
2019-10-16,233.37,235.24,233.2,234.37,1.84758e+07,-16.3478,60.813964,87.202899,16.903002,5.175551,0.283555,88.0,36.0,-0.121818,0.0,0.0,66.169156
2019-10-15,236.39,237.65,234.88,235.32,2.184e+07,-12.2174,60.940800,90.534597,16.323225,5.109180,0.270484,92.0,40.0,-0.082816,0.0,0.0,68.066117
2019-10-14,234.9,238.13,234.67,235.87,2.41069e+07,-9.82609,66.559863,93.845031,15.549671,4.853484,0.255657,96.0,4.0,-0.029347,1.0,0.0,69.131568
